In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models
import datetime

# 加载数据集并进行预处理
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
data_path = '../data-unversioned/p1ch7/'
trainset = datasets.CIFAR10(root=data_path, train=True,
                            download=True, transform=transform_train)
testset = datasets.CIFAR10(root=data_path, train=False,
                           download=True, transform=transform_test)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=4)


Files already downloaded and verified
Files already downloaded and verified


In [10]:
# 定义模型
model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
# model = model.cuda()

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# 训练模型
for epoch in range(50):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            s = '[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 100)
            print('{} {}'.format(datetime.datetime.now(),s))
            running_loss = 0.0

print('Training finished')


[1,   100] loss: 3.035
[1,   200] loss: 2.096
[1,   300] loss: 1.818
[2,   100] loss: 1.558
[2,   200] loss: 1.533
[2,   300] loss: 1.481
[3,   100] loss: 1.368
[3,   200] loss: 1.343
[3,   300] loss: 1.278
[4,   100] loss: 1.208
[4,   200] loss: 1.196
[4,   300] loss: 1.163
[5,   100] loss: 1.188
[5,   200] loss: 1.107
[5,   300] loss: 1.088
[6,   100] loss: 1.036
[6,   200] loss: 1.016
[6,   300] loss: 1.011
[7,   100] loss: 0.951
[7,   200] loss: 0.970
[7,   300] loss: 0.961
[8,   100] loss: 0.918
[8,   200] loss: 0.903
[8,   300] loss: 0.926
[9,   100] loss: 0.872
[9,   200] loss: 0.868
[9,   300] loss: 0.896
[10,   100] loss: 0.865
[10,   200] loss: 0.840
[10,   300] loss: 0.864
[11,   100] loss: 0.848
[11,   200] loss: 0.823
[11,   300] loss: 0.833
[12,   100] loss: 0.799
[12,   200] loss: 0.818
[12,   300] loss: 0.812
[13,   100] loss: 0.795
[13,   200] loss: 0.807
[13,   300] loss: 0.793
[14,   100] loss: 0.790
[14,   200] loss: 0.791
[14,   300] loss: 0.789
[15,   100] loss: 0

In [11]:
# 在测试集上评估模型性能
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on test set: %d %%' % (100 * correct / total))

Accuracy on test set: 75 %


In [17]:
# device = torch.device('cuda')
device = torch.device('cpu')

train_loader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False)
def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        with torch.no_grad():  # <1>
            for imgs, labels in loader:
                outputs = model(imgs.to(device))
                _, predicted = torch.max(outputs, dim=1) # <2>
                total += labels.to(device).shape[0]  # <3>
                correct += int((predicted == labels.to(device)).sum())  # <4>
        print("Accuracy {}: {:.2f}".format(name , correct / total))
print(datetime.datetime.now(), '开始运算')
validate(model, train_loader, val_loader)
print(datetime.datetime.now(), '结束运算')

2023-05-23 13:44:54.205549 开始运算


RuntimeError: Input type (MPSFloatType) and weight type (torch.FloatTensor) should be the same

In [16]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        with torch.no_grad():  # <1>
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <2>
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())  # <4>
        print("Accuracy {}: {:.2f}".format(name , correct / total))

print(datetime.datetime.now(), '开始运算')
validate(model, train_loader, val_loader)
print(datetime.datetime.now(), '结束运算')

2023-05-23 13:20:58.988616 开始运算
Accuracy train: 0.78
Accuracy val: 0.75
2023-05-23 13:30:09.954649 结束运算
